In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import qiime2 as q2
from qiime2 import Visualization
import seaborn as sns
from scipy.stats import shapiro, kruskal, f_oneway

div_dir = 'poop_data/Diversity'
phy_dir = 'poop_data/Phylogeny'
tox_dir = 'poop_data/Taxonomy'
data_dir = 'poop_data'

%matplotlib inline